# Multi-Output Neural Network

In [ ]:
# imports
import gzip
import numpy as np
import pandas as pd
import pandas_ml

import keras # Ignore deprecated warning
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import Sequential

In [ ]:
# FashionMNIST: https://github.com/zalandoresearch/fashion-mnist
def open_images(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=16)\
            .reshape(-1, 28, 28)\
            .astype(np.float32)


def open_labels(filename):
    with gzip.open(filename, "rb") as file:
        data = file.read()
        return np.frombuffer(data, dtype=np.uint8, offset=8)

In [ ]:
# Prepare data
x_train = open_images("../res/fashionmnist/train-images-idx3-ubyte.gz")
y_train = open_labels("../res/fashionmnist/train-labels-idx1-ubyte.gz")

x_test = open_images("../res/fashionmnist/t10k-images-idx3-ubyte.gz")
y_test = open_labels("../res/fashionmnist/t10k-labels-idx1-ubyte.gz")

In [ ]:
# Visualize data
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(x_train[1], cmap = "gray_r")
plt.show()

y_train_multi = to_categorical(y_train) # Train model which detects all classes
y_test_multi = to_categorical(y_test)

### Multi-Output Neural Network
... using softmax activation function.

In [ ]:
# Define neuronal network (multi output)
model = Sequential()

model.add( Dense(100, activation = "sigmoid", input_shape = (784,)) )
model.add(Dense(10, activation = "softmax"))

# Create efficient code (e.g. for GPU)
# Choose an optimizer: sgd, rmsprop, adam, 
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics = ["accuracy"]) 

In [ ]:
# Train model
model.fit(x_train.reshape(60000, 784),
          y_train_multi,
          epochs = 10,
          batch_size = 1000) 

In [ ]:
# Evaluate using TRAIN data
print(model.metrics_names)
print(model.evaluate(x_train.reshape(-1, 784), y_train_multi))

In [ ]:
# Predict an image
train_index = 0

# Predict value
prediction = np.argmax(model.predict(x_test.reshape(-1, 784))[train_index])

if prediction == 0:
    print('This is a t-shirt')
if prediction == 9:
    print('This is a shoe')
else:
    print('This is an other class')
    
# Show sample image
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(x_train[train_index], cmap = "gray_r")
plt.show()

### Confusion Matrix of Neural Network
Can be used to see which classes can be detected well and which ones are confused with others.

In [ ]:
# Predict test data
pred = model.predict(x_test.reshape(-1, 784))
pred = np.argmax(pred, axis = 1) # Find max. prop. of class in output array

In [ ]:
# Create confusion matrix
confusion_matrix = pandas_ml.ConfusionMatrix(pred, y_test)
print(confusion_matrix)

### Is my network complex enough?
Es gilt: Das Netz passt sich an die Daten an
- Testgenauigkeit nur minimal schlechter wie die Trainingsgenauigkeit: Das Modell darf vermutlich noch was komplexer sein
- Testgenauigkeit sehr viel schlechter wie die Trainingsgenauigkeit: Das Modell ist zu komplex (das Modell hat sich zu stark an die Trainingsdaten angepasst)
- Beides schlecht: Du brauchst mehr Daten oder einen anderen Algorithmus / Optimierer!

In [ ]:
# Define neuronal network (multi output) with additional layer
model = Sequential()

# Choose activation functions: tanh sigmoid relu softwax
model.add( Dense(100, activation = "tanh", input_shape = (784,)) )
model.add( Dense(50, activation = "sigmoid", input_shape = (784,)) )
model.add(Dense(10, activation = "softmax"))

# Create efficient code (e.g. for GPU)
# Choose optimizer
# rmsprop adam 
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy", metrics = ["accuracy"]) 

In [ ]:
# Train model
model.fit(x_train.reshape(60000, 784),
          y_train_multi,
          epochs = 10,
          batch_size = 1000) 

In [ ]:
# Evaluate using TRAIN data
print(model.metrics_names)
print(model.evaluate(x_test.reshape(-1, 784), y_test_multi))